In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Student Name: Arjav Chauhan
# Student ID: 0191086
# Assignment: Student Performance Data Set
# Assignment Link: https://archive.ics.uci.edu/ml/datasets/student+performance

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# path_math = "../input/student-performance-data-set/student/student-mat.csv"
path_port = "../input/student-performance-data-set/student/student-por.csv"

# math = pd.read_csv(path_math, sep=";")
port = pd.read_csv(path_port, sep=";")

In [ ]:
# Concatenating dataframes
data = pd.concat([port])

In [ ]:
len(data)

In [ ]:
data.head()

In [ ]:
# generate binary label (pass/fail) based on G1+G2+G3 (test grades, each 0 - 20 points); threshold for passing is sum >= 30
data['pass'] = data.apply(lambda row: 1 if (row['G1'] + row['G2'] + row['G3']) >= 35 else 0, axis = 1)
data = data.drop(['G1', 'G2', 'G3'], axis = 1)
data.head()

In [ ]:
# use one-hot encoding on categorical columns
data = pd.get_dummies(data, columns = ['sex', 'school', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 
                                    'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 
                                    'internet', 'romantic'])
data.head()

In [ ]:
# shuffle rows
data = data.sample(frac = 1)

# split training and testing data
data_train = data[:500]
data_test = data[500:]

data_train_att = data_train.drop(['pass'], axis = 1)
data_train_pass = data_train['pass']

data_test_att = data_test.drop(['pass'], axis = 1)
data_test_pass = data_test['pass']

data_att = data.drop(['pass'], axis = 1)
data_pass = data['pass']

# number of passing students in whole dataset:
print("Passing %d out of %d (%.2f%%)" % (np.sum(data_pass), len(data_pass), 100 * float(np.sum(data_pass)) / len(data_pass)))

In [ ]:
# fit a decision tree
from sklearn import tree
t = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = 5)
t = t.fit(data_train_att, data_train_pass)

# visualize tree
import graphviz
dot_data = tree.export_graphviz(t, out_file = None, label = "all", impurity = False, proportion = True, 
                                feature_names = list(data_train_att), class_names = ["fail", "pass"], 
                                filled = True, rounded = True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
# save tree
tree.export_graphviz(t, out_file = "student-performance.dot", label = "all", impurity = False, proportion = True, 
                     feature_names = list(data_train_att), class_names = ["fail", "pass"], 
                     filled = True, rounded = True)

In [ ]:
t.score(data_test_att, data_test_pass)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(t, data_att, data_pass, cv = 5)

# show average score and =/- two standard deviations away (covering 95% of scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
for max_depth in range(1, 20):
    t = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = max_depth)
    scores = cross_val_score(t, data_att, data_pass, cv = 5)
    print("Max Depth: %d, Accuracy: %0.2f (+/- %0.2f)" % (max_depth, scores.mean(), scores.std() * 2))

In [ ]:
depth_acc = np.empty((19, 3), float)
i = 0
for max_depth in range(1, 20):
    t = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = max_depth)
    scores = cross_val_score(t, data_att, data_pass, cv = 5)
    depth_acc[i, 0] = max_depth
    depth_acc[i, 1] = scores.mean()
    depth_acc[i, 2] = scores.std() * 2
    i += 1
    
depth_acc

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.errorbar(depth_acc[:, 0], depth_acc[:, 1], yerr = depth_acc[:, 2])
plt.show()